# Load Model and Pre-Processing Steps

* Label encoder
* Tabular transformer
* ResNet50 image featuriser
* Image PCA
* Fitted model of choice

In [ ]:
import pickle

# Load label encoder
with open("skin_cancer_encode_labels.preprocess", "rb") as file:
    label_encoder = pickle.load(file)
# Load tabular preprocessor
with open("skin_cancer_tabular.preprocess", "rb") as file:
    tabular_preprocessor = pickle.load(file)
# Load PCA for CNN
with open("skin_cancer_image_pca.preprocess", "rb") as file:
    image_pca = pickle.load(file)
# Load model
with open("ovr_en_logreg.model", "rb") as file:
    model = pickle.load(file)

In [ ]:
try:
    import tensorflow as tf
except:
    !pip install tensorflow
    import tensorflow as tf

# Load pre-trained CNN
pre_cnn_model = tf.keras.applications.ResNet50(
    include_top=False,  # Remove classification head
    input_shape=(85, 85, 3),  # Input shape that the model is most used to
    pooling='avg',      # Return vector
    weights='imagenet'  # Pre-trained weights
)
pre_cnn_model.trainable = False

# Check
# pre_cnn_model.summary()

def featurise_image(image):
    # Build complete CNN based on dimensions of input image
    # Not the most elegant but ensures most reproducibility compared to original pre-processing
    # Input layer is shape of NumPy array
    inputs = tf.keras.Input(shape=image.shape)
    # Crop into square - minimum of height, weight
    sq_size = min(image.shape[0], image.shape[1])
    cropped = tf.keras.layers.CenterCrop(sq_size, sq_size)(inputs)
    # Resize down
    resized = tf.keras.layers.Resizing(85, 85)(cropped)
    # Reorder RGB to BGR and normalise
    scaled = tf.keras.applications.resnet.preprocess_input(resized)
    # Get features using CNN model
    output_features = pre_cnn_model(scaled)
    # Compile models with these layers
    image_featuriser = tf.keras.Model(inputs, output_features)
    image_featuriser.trainable = False

    # CNN model expects batches, so add extra dimension of size 1
    image_np = np.expand_dims(image, axis=0)
    # Featurise and return
    return image_featuriser(image_np, training=False).numpy()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


# Demo

* Define function to generate predictions
* Use Gradio to wrap UI around that function

In [ ]:
import pandas as pd
import numpy as np

# Define function to get prediction from inputs
def predict_dx(image, age, sex, localization):
    # Process tabular data
    tabular_df = pd.DataFrame([[age,sex,localization]], columns=["age","sex","localization"])
    # Transform
    tabular_np = tabular_preprocessor.transform(tabular_df)

    # Get features from image
    image_features_np = featurise_image(image)
    # Apply PCA
    image_features_np = image_pca.transform(image_features_np)

    # Concatenate features together
    all_features_np = np.concatenate((image_features_np, tabular_np), axis=1)

    # Get prediction
    prediction = model.predict(all_features_np)
    # If prediction is 2-D array, it means returned probabilities for classes so apply argmax
    if prediction.ndim > 1:
        prediction = np.argmax(prediction, axis=1)
    # Now have label, inverse transform with label encoder to get label name
    return label_encoder.inverse_transform(prediction)[0]

In [ ]:
import gradio as gr

# Close previous Gradio things to be sure
gr.close_all()

# Get sex categories as one-hot encoder knows them
sex_values = tabular_preprocessor.named_transformers_['cat'].categories_[0]
# Same with localisation
local_values = tabular_preprocessor.named_transformers_['cat'].categories_[1]

demo = gr.Interface(
    fn=predict_dx,
    inputs=[gr.Image(),
            gr.Slider(minimum=0, maximum=85, step=1),
            gr.Radio(sex_values.tolist()),
            gr.Radio(local_values.tolist())
            ],
    outputs=["text"],
)

demo.launch()

Closing server running on port: 7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6e61f8ba8444bf0151.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
